In [132]:
import pandas as pd
from Transaction import *
from nacl.signing import SigningKey
from nacl.encoding import Base64Encoder
from hashlib import sha256
from nacl.encoding import HexEncoder
from hashlib import sha256 as H

In [133]:
# create the keys first using a dataframe 
keys_list = pd.DataFrame()

In [134]:
# generate a list of 8 public keys and 8 private keys along with them and the signatures along with them since the signatures should stay the same for each public key
pub_keys_list = []
secret_keys_list = []
users_list = []
user = 'user'
for x in range(0, 9):
    users_list.append(user + str(x))
    sk = SigningKey.generate()
    secret_keys_list.append(sk)
    pk = sk.verify_key
    pub_keys_list.append(pk)

In [135]:
# add these two list to the dataframe
keys_list['users'] = users_list
keys_list['pub_key'] = pub_keys_list
keys_list['sk'] = secret_keys_list

In [136]:
print(keys_list)

   users                                            pub_key  \
0  user0  b"k\xa6\x97F\x1f\x0e|&\xa2\xe9\xb9\x84\xbevF\x...   
1  user1  b'\x19\xcc\x1d\xd5r\xdfL\x90\x1d~\xe2I4\x12\r\...   
2  user2  b"2\x9c\xed\x9f!\xf9\xd6\xed\xfc|\x88N\xfd\x8a...   
3  user3  b'\xed\xa7\xcdQ\xb6\x07G\xecp\xe8\xf8\x94\x95\...   
4  user4  b'\x08F\x0f\x91\xd9\x15!f\x0b\x18\xc3\x86\x9e3...   
5  user5  b'\xa7\x95e\xed\x96\xc8\xd3\xc8\x94\xf0\xb6\xa...   
6  user6  b'\xb715\xd7\xf7\x01m\x13k\x03\x8a\xac+~w%d\xa...   
7  user7  b'\xd1w\xadR\\z\x12\x05\x9bj\xa6\xc1\xb8FR\x99...   
8  user8  b'c\xe6o%Ro-@\xc2\xf9\xfegt|\xd6\xbd;{1Y\xe0\x...   

                                                  sk  
0  b"\xfd\xbc\x8d\xbc@,\x19\x19~''\xfe\x02\x8f\xa...  
1  b'\x96x\n,\xf6p\xd8`\xff\x17@\x83\x93\x8ak\xc8...  
2  b'\x9a\xa9\xf53\xab\xb2B\xcd\xddxKS\x1aae\x92\...  
3  b'\xd9\x84\xd3N(8ra\xab\xe2FKF\x97"\xea\x8b\xa...  
4  b'j\x02\x0em\xd8\xf1\xcc8\xc5v=\xd8\xdc\xfaW\x...  
5  b'\xccJ\xe0\xde<\xf5\xba\xe2\xffFF\x

In [137]:
from nacl.signing import SigningKey

def createSignature(input, output, user_sk):
    # temp = input.encode()
    temp = output.encode()
    signature = user_sk.sign(temp, encoder=Base64Encoder).decode()
    return signature

In [138]:
# create the genesis transaction first - create 8 different ones 
output_1 = transaction_output(100, keys_list['pub_key'][0].encode(encoder=Base64Encoder).decode())
new_output_1 = output_1.create_list()
output_2 = transaction_output(100, keys_list['pub_key'][1].encode(encoder=Base64Encoder).decode())
new_output_2 = output_2.create_list()
output_3 = transaction_output(100, keys_list['pub_key'][2].encode(encoder=Base64Encoder).decode())
new_output_3 = output_3.create_list()
output_4 = transaction_output(100, keys_list['pub_key'][3].encode(encoder=Base64Encoder).decode())
new_output_4 = output_4.create_list()
output_5 = transaction_output(100, keys_list['pub_key'][4].encode(encoder=Base64Encoder).decode())
new_output_5 = output_5.create_list()
output_6 = transaction_output(100, keys_list['pub_key'][5].encode(encoder=Base64Encoder).decode())
new_output_6 = output_6.create_list()
output_7 = transaction_output(100, keys_list['pub_key'][6].encode(encoder=Base64Encoder).decode())
new_output_7 = output_7.create_list()
output_8 = transaction_output(100, keys_list['pub_key'][7].encode(encoder=Base64Encoder).decode())
new_output_8 = output_8.create_list()

In [139]:
all_outputs = []
all_outputs.append(new_output_1)
all_outputs.append(new_output_2)
all_outputs.append(new_output_3)
all_outputs.append(new_output_4)
all_outputs.append(new_output_5)
all_outputs.append(new_output_6)
all_outputs.append(new_output_7)
all_outputs.append(new_output_8)


In [140]:
g_sig = createSignature(None, json.dumps(all_outputs), keys_list['sk'][0])

In [141]:
def value(transaction_number, output_list, sig):
    list_value = [str(transaction_number)]
    for output_tx in output_list:
        list_value.append(output_tx.toString())
    list_value.append(sig)
    return ''.join(list_value)

In [142]:
g_transaction = Transaction(1, None, all_outputs, g_sig, None)
json_object = json.dumps(g_transaction.getJson(1), indent=4)
with open("genesis_tx.json", "w") as outfile:
    outfile.write(json_object)

In [143]:
new_dataframe = pd.DataFrame()
pubkey = []
skey = []
for x in range(0, 9):
    users_list.append(user + str(x))
for i in keys_list['pub_key']:
    pk = i.encode(encoder=Base64Encoder).decode()
    pubkey.append(pk)
for x in keys_list['sk']:
    sk = x.encode(encoder=Base64Encoder).decode()
    skey.append(sk)
new_dataframe['pubkey'] = pubkey
new_dataframe['skey'] = skey
new_dataframe.to_csv('keys.csv')

In [144]:
my_key_dict = new_dataframe.set_index('pubkey').T.to_dict('skey')

/var/folders/vt/v9235t1j1fz10ydwp5g_5s7c0000gn/T/ipykernel_39640/1065302050.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  my_key_dict = new_dataframe.set_index('pubkey').T.to_dict('skey')
